In [1]:
import csv
f = open("input.txt") # Default read
print(f.read())

10|1|SELL|toaster_1|10.00|20
12|8|BID|toaster_1|7.50
13|5|BID|toaster_1|12.50
15|8|SELL|tv_1|250.00|20
16
17|8|BID|toaster_1|20.00
18|1|BID|tv_1|150.00
19|3|BID|tv_1|200.00
20
21|3|BID|tv_1|300.00



In [2]:
# Dummy Data classification function to check on input lines
def dataClass(data=[]):
    print(data)

In [3]:
#read lines and classify
# classify type : Sell Buy Heartbeat
def readAndClass(input=""):
    with open(file=input, newline='') as csvFile:
        csvLines = csv.reader(csvFile, delimiter='|')
        list(map(dataClass,csvLines))
readAndClass(input="input.txt")

['10', '1', 'SELL', 'toaster_1', '10.00', '20']
['12', '8', 'BID', 'toaster_1', '7.50']
['13', '5', 'BID', 'toaster_1', '12.50']
['15', '8', 'SELL', 'tv_1', '250.00', '20']
['16']
['17', '8', 'BID', 'toaster_1', '20.00']
['18', '1', 'BID', 'tv_1', '150.00']
['19', '3', 'BID', 'tv_1', '200.00']
['20']
['21', '3', 'BID', 'tv_1', '300.00']


In [4]:
from collections import namedtuple
bids = namedtuple('bidItems', 'timestamp user_id action item bid_amount')
item = namedtuple('items', 'timestamp user_id action item reserve_price close_time')
auction_order = []
hearbeat = [] #Timestamps

In [5]:
# from collections import defaultdict
# biddings = defaultdict(list) # create biddings dictionary
# items = {} # set to remove any duplicate entries/items
# auction_order = []

In [6]:
#update data classification function here
def dataClass(data=[]):
    # print(data)
    if len(data) == 1: # heartbeat
        hearbeat.append(int(data[0]))
        print("Heartbeat")
    elif len(data) == 5: # Bid
        auction_order.append(bids._make(data)) # generate new bids from data
        print("Bid")
    elif len(data) == 6: #Sell
        auction_order.append(item._make(data)) # generate new item from data
        print("List")

In [7]:
# test input reading and classification
readAndClass(input='input.txt')

List
Bid
Bid
List
Heartbeat
Bid
Bid
Bid
Heartbeat
Bid


In [8]:
# After classifying, need to find a solution to process the 'auction_orders' list
#before that set a class for auction items
class AuctionItems:
    def __init__(self, name, data):
        self.name = name
        self.data = data
        self.all_bids = []
        self.price_paid = 0.0 #should be the price paid by the auction winner (0.00 if the item is UNSOLD), as a number to two decimal places
        self.status = 'UNSOLD' # 'SOLD' or 'UNSOLD depending on auction outcome, default 'UNSOLD'
        self.total_bid_count = 0 # number of 'valid' bids received for the item (validity will be addressed later)
        self.highest = 0.0 # The highest bid received for the item as a number to 2 dp
        self.lowest = 0.0 # The Lowest bid placed on the item as a number to 2 dp
        self.bids = namedtuple('bidItems', 'timestamp user_id action item bid_amount')

    def get_name(self):
        return self.name

    def get_data(self):
        return self.data

    def addAllBids(self, bid):
        if isinstance(bid, self.bids):
            self.all_bids.append(bid)

    def setPricePaid(self,price):
        self.price_paid = price

    def setHighestBid(self,price):
        self.highest = price if price >= self.all_bids[-1] else 0

    # def setLowestBid(self, price):
    #
    #     self.lowest = price if price <= self.lowest else self.lowest

    def setStatus(self,status):
        if status == 'SOLD' or status == "UNSOLD":
            self.status = status

    def incrementBidCount(self):
        self.total_bid_count += 1

    def getStatus(self):
        return self.status

    def get_bids(self):
        return self.all_bids

    def getFullData(self): #this matches with the output requried
        return "{0}|{1}|{2}|{3}|{4}|{5}|{6}|{7}".format(self.data.close_time,
                                self.name,
                                self.data.user_id,
                                self.status,
                                self.price_paid,
                                self.total_bid_count,
                                self.highest,
                                self.lowest)

`close_time` should be a unix epoch of the time the auction finished
`item` is the unique string item code.
`user_id` is the integer id of the winning user, or blank if the item did not sell.
`status` should contain either "SOLD" or "UNSOLD" depending on the auction outcome.
`price_paid` should be the price paid by the auction winner (0.00 if the item is UNSOLD), as a
number to two decimal places
`total_bid_count` should be the number of valid bids received for the item.
'highest_bid' the highest bid received for the item as a number to two decimal places
`lowest_bid` the lowest bid placed on the item as a number to two decimal places

In [9]:
#Continuing to process the 'Auction_orders' list
all_auction_items = {}
closing_times = {}
def processOrders(instructions):
    if isinstance(instructions, item):
        all_auction_items[instructions.item] = AuctionItems(name=instructions.item, data=instructions)
        closing_times[instructions.item] = instructions.close_time
        print("item added")
    elif isinstance(instructions,bids):
        price = bids.bid_amount
        if instructions.item in all_auction_items.keys():
            auction_item = all_auction_items[instructions.item]
            bid_instructions = auction_item.all_bids
            # try:
                #check valid [delete]
            if not bid_instructions: # if bid instruction is empty, that means its the first one
                auction_item.lowest = price
            auction_item.addAllBids(instructions)
            auction_item.incrementBidCount()
            # except Exception as bidError:
            #     print("Error when adding bid")
            print("bid successful")
        else:
            print("Item not auctioned yet")
    # Heartbeat to be added afterwards
    elif isinstance(instructions,hearbeat):
        print("heartbeat")
    #     heartbeat = instructions.timestamp
    #
    else:
        print("invalid auction instruction")

list(map(processOrders,auction_order))

item added
bid successful
bid successful
item added
bid successful
bid successful
bid successful
bid successful


[None, None, None, None, None, None, None, None]

In [10]:
# now check on the following validities:
# current timestamp strictly larger than the last one and placed within auctioning time
# current bid is strictly greater than the last bid of the item, and the bid is also larger than the reserve bid price


In [11]:
# current timestamp strictly larger than the last one and placed within auctioning time
from decimal import *
def checkTime(instr_timestamp, item):
    check = True # initialise

    #First check the timestamp ( This may be useless but just a precaution check)
    if item.all_bids: # if there are existing bids in the list, else skip as theres no point comparing an empty list, it just gives you an error :)
        if Decimal(instr_timestamp) > Decimal(item.all_bids[-1].timestamp): #cast to decimal for comparison purposes
            check &= True
            print("Auction instruction placed in a reasonable time")
        else:
            check &= False
            print("Auction instruction NOT placed in a reasonable time")

    # Now check if its within auction time, if its False from the last one, do nothing, its already invalid
    if check:
        if Decimal(instr_timestamp) < Decimal(item.data.close_time):
            check &= True
            print("Auction instruction is within item close time")
        else:
            check &= False
            print("Auction instruction is out of item close time")

    #Code alternatives : reverse the decimal comparison, and reverse the check &(s) so that else can be removed, same but depends what looks cleaner I guess.

    return check

In [12]:
# current bid is strictly greater than the last bid of the item, and the bid is also larger than the reserve bid price
def checkPrice(price, item):
    #check if current bid is strictly greater than the last bid of the item
    check = True # initialise
    if item.all_bids:
        if Decimal(price) > Decimal(item.all_bids[-1].bid_amount):
            check &= True
            print("Price placed is larger than the last bid price")
        else:
            check &= False
            print("Price placed is smaller or equal to the last bid price")
    #the bid is also larger than the reserve bid price
    if check:
        if Decimal(price) >= Decimal(item.data.reserve_price):
            check &= True
            print("Price is larger than the reserve price")
        else:
            check &= False
            print("Price is smaller than the reserve price")

    return check

In [13]:
def validBidAction(bid):  #'timestamp user_id action item bid_amount'
    check = True
    if all_auction_items: # if list of items in the auction is not empty
        if bid.item in all_auction_items.keys(): # if item is added to the auction already
            if checkPrice(bid.bid_amount, all_auction_items[bid.item]) and checkTime(bid.timestamp, all_auction_items[bid.item]):
                check &= True
                print("valid bet")
            else:
                check &= False
                print("invalid price or timestamp")
        else:
            print("invalid bet, item not in auction")
            check = False
    else:
        print("No items is under auction yet")
        check = False
    return check


In [ ]:
# One last thing is to deal with the 'heartbeat' element which relates to closing the auction

In [ ]:
def processheartBeats(item_name):
    item = all_auction_items[item_name]
    if item.all_bids:
        if len(item.all_bids) >= 2: #based on example output, we return the 2nd highest bid price
            item.setPricePaid(item.all_bids[-2].bid_amount)
            # item.setStatus('SOLD')
        else:
            item.setPricePaid(item.reserve_price)
        item.setStatus('SOLD')
        print("item {0} is sold for {1}".format(item.name, item.pricePaid))
    else:
        item.setPricePaid(0.0)
        item.setStatus('UNSOLD')
        print("item {0} is not sold, item is now closed for auction".format(item.name))

